In [1]:
! pip install faiss-cpu==1.7.4 mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 12.5 MB/s eta 0:00:00


In [18]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= getpass("Type your API Key")
client = Mistral(api_key=api_key)

Type your API Key··········


In [15]:
model = "mistral-large-latest"
chat_response = client.chat.complete(
    model = model,
    messages = [
        {
            "role": "user",
            "content": "What is the best French cheese?",
        },
    ]
)

print(chat_response.choices[0].message.content)

France is renowned for its diverse and delicious cheeses, and determining the "best" one can be subjective, as it often depends on personal taste. However, there are several French cheeses that are highly acclaimed worldwide. Here are a few notable ones:

1. **Camembert de Normandie**: A soft, creamy cheese from the Normandy region, famous for its rich flavor and melting texture.
2. **Roquefort**: A blue cheese made from sheep's milk, known for its strong, tangy flavor and characteristic veins of blue mold.
3. **Brie de Meaux**: Often referred to as the "King of Cheeses," this soft cheese has a rich, buttery flavor and a creamy texture.
4. **Comté**: A hard cheese made from unpasteurized cow's milk, known for its complex, nutty flavor and firm yet supple texture.
5. **Reblochon**: A soft washed-rind and smear-ripened cheese from the Alps, popular for its mild, nutty flavor and melting qualities, often used in tartiflette.
6. **Epoisses**: A pungent, soft cheese with a washed rind, know

In [3]:
response = requests.get('https://gist.githubusercontent.com/shawwn/1b399325e8667311654cee583b162e73/raw/7a4b28c84f3db9df7bc5ea8215fa64d4ea2a412f/apology.txt')
text = response.text

In [4]:
text

'A  \nMATHEMATICIAN\'S  \nAPOLOGY\n\n# A  \nMATHEMATICIAN\'S  \nAPOLOGY\n\nBY\n\nG. H. HARDY\n\nWITH A FOREWORD BY\n\nC. P. SNOW\n\nCAMBRIDGE UNIVERSITY PRESS\n\nCambridge, New York, Melbourne, Madrid, Cape Town, Singapore, São Paulo,\n\nDelhi, Mexico City\n\nCambridge University Press\n\nThe Edinburgh Building, Cambridge CB2 8RU, UK\n\nPublished in the United States of America by Cambridge University Press, New York\n\nwww.cambridge.org\n\nInformation on this title: www.cambridge.org/9781107604636\n\nForeword © C. P. Snow 1967\n\nThis publication is in copyright. Subject to statutory exception and to the provisions of relevant collective licensing agreements, no reproduction of any part may take place without the written permission of Cambridge University Press.\n\nFirst edition 1940  \nReprinted 1941, 1948  \nReprinted with Foreword  \nby C. P. Snow 1967, 1969,  \n1973, 1976, 1977, 1979, 1981,  \n1982, 1984, 1985, 1987, 1988,   \n1989, 1990\n\nCanto edition 1992\n\n19th printing 2012

In [5]:
f = open('apology.txt', 'w')
f.write(text)
f.close()

In [6]:
len(text)

166055

In [36]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

82

In [37]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings.create(
          model="mistral-embed",
          inputs=input
      )
    return embeddings_batch_response.data[0].embedding

In [38]:
import time
from tqdm import tqdm
#text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])
text_embeddings = []

for chunk in tqdm(chunks):
  time.sleep(10)
  text_embeddings.append(get_text_embedding(chunk))
text_embeddings = np.array(text_embeddings)

100%|██████████| 82/82 [14:35<00:00, 10.67s/it]


In [39]:
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [49]:
question = "What were the two main things the author worked on before college?"
question_embeddings = np.array([get_text_embedding(question)])

In [50]:
D, I = index.search(question_embeddings, k=2) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

In [51]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [52]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

run_mistral(prompt)

'Based on the provided context, the author does not explicitly mention two main things they worked on before college. However, they do mention that their early ambitions in mathematics were focused on "examinations and scholarships" and that they wanted to "beat other boys" in these areas. Therefore, the two main things the author was involved with before college were preparing for examinations and competing for scholarships.'

In [53]:
question = "What were the two main things the author write in the book?"
question_embeddings = np.array([get_text_embedding(question)])

In [54]:
D, I = index.search(question_embeddings, k=2) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

In [55]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [57]:
answer = run_mistral(prompt)
answer

'Based on the provided context, the author writes about two main things in the book:\n\n1. **The Nature of a Mathematician\'s Work**: The author compares a mathematician to a painter or a poet, describing them as a maker of patterns. However, a mathematician\'s patterns are made with ideas, which are more permanent than the shapes, colors, or words used by painters or poets.\n\n2. **Personal Journey into Mathematics**: The author shares their personal journey and ambitions in mathematics, which started from a desire to beat other boys in examinations and scholarships. This autobiographical section includes a formative experience inspired by a book called "A Fellow of Trinity" by Alan St Aubyn.'

In [58]:
print(answer)

Based on the provided context, the author writes about two main things in the book:

1. **The Nature of a Mathematician's Work**: The author compares a mathematician to a painter or a poet, describing them as a maker of patterns. However, a mathematician's patterns are made with ideas, which are more permanent than the shapes, colors, or words used by painters or poets.

2. **Personal Journey into Mathematics**: The author shares their personal journey and ambitions in mathematics, which started from a desire to beat other boys in examinations and scholarships. This autobiographical section includes a formative experience inspired by a book called "A Fellow of Trinity" by Alan St Aubyn.


In [63]:
question = "What is the most populat quote from this book(Math Apology)?"
question_embeddings = np.array([get_text_embedding(question)])
D, I = index.search(question_embeddings, k=2) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [64]:
answer = run_mistral(prompt)
answer

'The context provided does not include specific quotes from "A Mathematician\'s Apology" by G.H. Hardy. However, one of the most popular quotes from this book is often cited as:\n\n"A mathematician, like a painter or a poet, is a maker of patterns. If his patterns are more permanent than theirs, it is because they are made with ideas."\n\nThis quote encapsulates Hardy\'s view on the aesthetic and creative nature of mathematics.'

In [65]:
print(answer)

The context provided does not include specific quotes from "A Mathematician's Apology" by G.H. Hardy. However, one of the most popular quotes from this book is often cited as:

"A mathematician, like a painter or a poet, is a maker of patterns. If his patterns are more permanent than theirs, it is because they are made with ideas."

This quote encapsulates Hardy's view on the aesthetic and creative nature of mathematics.
